In [2]:
import glob
import logging
import os
import pickle
import random
import re
import shutil
import copy
from typing import Dict, List, Tuple

import nltk
import numpy as np
import torch
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader, Dataset, RandomSampler, SequentialSampler
from torch.utils.data.distributed import DistributedSampler
from tqdm import tqdm, trange

from transformers import (MODEL_WITH_LM_HEAD_MAPPING, WEIGHTS_NAME, AdamW, AutoConfig, AutoModelWithLMHead, AutoTokenizer, PreTrainedModel, PreTrainedTokenizer, get_linear_schedule_with_warmup)

try:
    from torch.utils.tensorboard import SummaryWriter
except ImportError:
    from tensorboardX import SummaryWriter

In [3]:
model_type='bert'
model_name_or_path='bert-base-uncased'
cache_dir=None

In [30]:
def open_file(file_path):
    with open(file_path) as f:
        output_file=f.read().splitlines()

    return output_file

def encode_all_lines(data):
    encoded_data=[]
    segments=[]
    for i in range(len(data)):
        encoded_data.append(torch.tensor(tokenizer.encode(data[i], add_special_tokens=True)))
    return encoded_data, segments

In [5]:
gender_attr=open_file('./attribute_target_words/attributes/gender_new.txt')
occupation_stereo_target=open_file('./attribute_target_words/targets/gender_occupation.txt')

In [8]:
config=AutoConfig.from_pretrained(model_name_or_path, cache_dir=cache_dir)
config.output_hidden_states='true'

tokenizer=AutoTokenizer.from_pretrained(model_name_or_path, cache_dir=cache_dir)

block_size=tokenizer.model_max_length

model=AutoModelWithLMHead.from_pretrained(model_name_or_path, from_tf=bool(".ckpt" in model_name_or_path), config=config, cache_dir=cache_dir)

c:\users\31631\appdata\local\programs\python\python39\lib\site-packages\transformers\models\auto\modeling_auto.py:803: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a

In [31]:
gender_attr_data=encode_all_lines(gender_attr)
occu_stereo_target_data=encode_all_lines(occupation_stereo_target)

In [25]:
model.eval()

BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=Tr

In [29]:
with torch.no_grad():
    outputs=model(gender_attr_data[0])

ValueError: not enough values to unpack (expected 2, got 1)